# Example of using ClimateGPT LLM with the document 

## "Strengthening and Democratizing the U.S. Artificial Intelligence Innovation Ecosystem: An Implementation Plan for a National Artificial Intelligence Research Resource" (NAIRR-TF-Final-Report-2023.pdf)

#### Examples of questions:

* List the facilities that a NAIRR resource should provide with respect to (a) computing, (b) AI/ML models, and (c) data.  
* What committees does the report refer to?  
* What are NAIRR's goals with respect to human capital?  
* What task forces are recommended in the report?  

In [ ]:
import requests
import os
import langchain
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain_community.document_loaders import PyPDFLoader

MODEL = 'eci-io/climategpt-7b'
os.environ['OPENAI_API_BASE'] = 'http://fc-api-server:8000/v1'
os.environ['OPENAI_API_KEY'] = 'EMPTY'

### Test that LLM API is up

In [ ]:
!curl  -X 'GET' 'http://fc-api-server:8000/v1/models' -H 'accept: application/json'

### Load PDF file and set up the model

In [ ]:
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")
file = 'NAIRR-TF-Final-Report-2023.pdf'
loader = PyPDFLoader(file)
index = VectorstoreIndexCreator(embedding=embedding).from_loaders([loader])
llm = ChatOpenAI(model=MODEL)

### The conversation code:

In [ ]:
def save_answer(question, answer):
    with open('output.txt', 'a') as file:
        file.write(question+'\n')
        file.write(answer+'\n\n')
    

def question(question):
    answer = index.query(question, llm=llm)
    
    print(f"\nAnswer: {answer}")

    save = input("\n\nWould you like to save your answer? (y/n): ")
    print('\n')
    if save=='y':
        save_answer(question, answer)

def conversation():
    while True:
        user_input = input("Question ('q' to quit): ")
        if user_input=='q':
            break
        question(user_input)

### Run the conversation:

In [ ]:
conversation()